In [1]:
import sys 
sys.path.append(rf"/nfs/home/genovese/thesis-wildfire-genovese/src")
from importlib import reload
import utils
reload(utils)
from utils import *
data_folder = "/nfs/home/genovese/thesis-wildfire-genovese/data/"

In [26]:
confini = gpd.read_file(data_folder+'clean_data/confini_piemonte/confini_piemonte.shp')
confini.head()

,cartodb_id,comune,Codice Com,geometry
0,838,sanmartinoalfieri,A005099,"POLYGON ((8.10299 44.83612, 8.10399 44.83548, ..."
1,964,montecastello,A006105,"POLYGON ((8.68521 44.9705, 8.68913 44.97299, 8..."
2,667,priola,A004177,"POLYGON ((8.01499 44.279, 8.01754 44.27937, 8...."
3,341,civiasco,A002043,"POLYGON ((8.32285 45.83257, 8.32924 45.82994, ..."
4,7475,occhieppoinferiore,A096040,"POLYGON ((8.0254 45.55869, 8.02695 45.55694, 8..."


In [30]:
df = pd.read_csv(rf'/nfs/home/genovese/thesis-wildfire-genovese/data/IFC/Mappa_ifc_2018.csv', encoding='utf-16', sep='\t')
df.head()

,Comune,Comune_shp,Regione,Anno,Aree Interne,Classe ampiezza demografca 21 (estesa),Descrizione Decile,Et amp demografica estesa,Et Grado Urb,et_zalt,Latitudine (generata),Longitudine (generata),Max. Decili,Comuni,Regioni
0,Acceglio,Acceglio,Piemonte,2.018,F - Ultraperiferico,1.1,Bassa,fino a 1000 abitanti,Zone rurali,Montagna interna,"44,482736363","6,956697063",3,Polygon,NaN
1,Acqui Terme,Acqui Terme,Piemonte,2.018,D - Intermedio,2.2,Bassa,10.001-50.000,Piccole città e sobborghi,Collina interna,"44,681576393","8,452644480",3,Polygon,NaN
2,Agliano Terme,Agliano Terme,Piemonte,2.018,C - Cintura,1.2,Lieve,1.001-5.000,Zone rurali,Collina interna,"44,786294850","8,249966674",5,Polygon,NaN
3,Agliè,AgliÃ¨,Piemonte,2.018,C - Cintura,1.2,Bassa,1.001-5.000,Piccole città e sobborghi,Collina interna,"45,364452541","7,776246339",3,Polygon,NaN
4,Agrate Conturbia,Agrate Conturbia,Piemonte,2.018,C - Cintura,1.2,Molto bassa,1.001-5.000,Zone rurali,Collina interna,"45,668681051","8,571362659",2,Polygon,NaN


In [39]:
from shapely.geometry import Point
df['geometry'] = [Point(x, y) for x,y in zip(df['Longitudine (generata)'].apply(lambda x: x.replace(',','.')).astype('float64'), df['Latitudine (generata)'].apply(lambda x: x.replace(',','.')).astype('float64'))]
gdf = gpd.GeoDataFrame(df)

In [59]:
gdf = gdf.set_crs(confini.crs)

def search_polygon(confini, point):
    if len(confini[point.within(confini)]) > 0:
        return confini[point.within(confini)].iloc[0]
    else:
        return None

ifc_2018 = gdf.copy()
ifc_2018['geometry'] = gdf['geometry'].apply(lambda x: search_polygon(confini.geometry, x))
ifc_2018.head()

,Comune,Comune_shp,Regione,Anno,Aree Interne,Classe ampiezza demografca 21 (estesa),Descrizione Decile,Et amp demografica estesa,Et Grado Urb,et_zalt,Latitudine (generata),Longitudine (generata),Max. Decili,Comuni,Regioni,geometry
0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,"42,700522028","12,127281188",NaN,NaN,MultiPolygon,None
1,Acceglio,Acceglio,Piemonte,2.019,F - Ultraperiferico,1.1,Moderata,fino a 1000 abitanti,Zone rurali,Montagna interna,"44,482736363","6,956697063",6.0,Polygon,NaN,"POLYGON ((6.92215 44.55494, 6.92492 44.55153, ..."
2,Acqui Terme,Acqui Terme,Piemonte,2.019,D - Intermedio,2.2,Bassa,10.001-50.000,Piccole città e sobborghi,Collina interna,"44,681576393","8,452644480",3.0,Polygon,NaN,"POLYGON ((8.43848 44.71089, 8.43929 44.70894, ..."
3,Agliano Terme,Agliano Terme,Piemonte,2.019,C - Cintura,1.2,Lieve,1.001-5.000,Zone rurali,Collina interna,"44,786294850","8,249966674",5.0,Polygon,NaN,"POLYGON ((8.23968 44.81115, 8.24013 44.81052, ..."
4,Agliè,AgliÃ¨,Piemonte,2.019,C - Cintura,1.2,Molto bassa,1.001-5.000,Piccole città e sobborghi,Collina interna,"45,364452541","7,776246339",2.0,Polygon,NaN,"POLYGON ((7.78266 45.38717, 7.79106 45.3863, 7..."


In [60]:
df = pd.read_csv(rf'/nfs/home/genovese/thesis-wildfire-genovese/data/IFC/Mappa_ifc_2019.csv', encoding='utf-16', sep='\t')
df['geometry'] = [Point(x, y) for x,y in zip(df['Longitudine (generata)'].apply(lambda x: x.replace(',','.')).astype('float64'), df['Latitudine (generata)'].apply(lambda x: x.replace(',','.')).astype('float64'))]
gdf = gpd.GeoDataFrame(df)
gdf = gdf.set_crs(confini.crs)

ifc_2019 = gdf.copy()
ifc_2019['geometry'] = gdf['geometry'].apply(lambda x: search_polygon(confini.geometry, x))

df = pd.read_csv(rf'/nfs/home/genovese/thesis-wildfire-genovese/data/IFC/Mappa_ifc_2021.csv', encoding='utf-16', sep='\t')
df['geometry'] = [Point(x, y) for x,y in zip(df['Longitudine (generata)'].apply(lambda x: x.replace(',','.')).astype('float64'), df['Latitudine (generata)'].apply(lambda x: x.replace(',','.')).astype('float64'))]
gdf = gpd.GeoDataFrame(df)
gdf = gdf.set_crs(confini.crs)

ifc_2021 = gdf.copy()
ifc_2021['geometry'] = gdf['geometry'].apply(lambda x: search_polygon(confini.geometry, x))

In [62]:
lst = [ifc_2018, ifc_2019, ifc_2021]

---

In [72]:
df = pd.read_csv(rf'/nfs/home/genovese/thesis-wildfire-genovese/data/IFC/tavole_comuni_2018_valore.csv', encoding='utf-16', sep='\t')

In [78]:
df[df.Comune.isna()]['Comune'] = 'None Torinese'

/tmp/ipykernel_1087574/1475043635.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df[df.Comune.isna()]['Comune'] = 'None Torinese'


In [80]:
lst[0][lst[0].Comune.isna()]

,Comune,Comune_shp,Regione,Anno,Aree Interne,Classe ampiezza demografca 21 (estesa),Descrizione Decile,Et amp demografica estesa,Et Grado Urb,et_zalt,Latitudine (generata),Longitudine (generata),Max. Decili,Comuni,Regioni,geometry
0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,"42,700522028","12,127281188",NaN,NaN,MultiPolygon,None
688,NaN,NaN,Piemonte,2.019,C - Cintura,2.1,Medio bassa,5.001-10.000,Piccole città e sobborghi,Pianura,"44,942328067","7,552195388",4.0,Polygon,NaN,"POLYGON ((7.55889 44.98047, 7.55698 44.97643, ..."


In [81]:
lst[0].loc[688, 'Comune'] = 'None Torinese'

In [83]:
df2018 = df.merge(lst[0], on='Comune', how='inner')

In [88]:
df = pd.read_csv(rf'/nfs/home/genovese/thesis-wildfire-genovese/data/IFC/tavole_comuni_2019_valore.csv', encoding='utf-16', sep='\t')

In [91]:
df2019 = df.merge(lst[1], on='Comune', how='inner')

In [92]:
df = pd.read_csv(rf'/nfs/home/genovese/thesis-wildfire-genovese/data/IFC/tavole_comuni_2021_valore.csv', encoding='utf-16', sep='\t')

In [94]:
df[df.Comune.isna()]['Comune'] = 'None Torinese'

/tmp/ipykernel_1087574/1475043635.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df[df.Comune.isna()]['Comune'] = 'None Torinese'


In [95]:
df2021 = df.merge(lst[2], on='Comune', how='inner')

---

In [98]:
lstdf = [df2018, df2019, df2021]

In [ ]:
# for df in lstdf:
#     df.drop(columns = df.columns[[1, 2, 3, 4, 5, 6, 20, 21, 22, 23, 28, 29, 30, 31, 32, 33]], inplace=True)
#     df = gpd.GeoDataFrame(df)

In [130]:
lstdf[0].loc[164, lstdf[0].columns[-5:]]

Classe ampiezza demografca 21 (estesa)     NaN
Descrizione Decile                         NaN
Et amp demografica estesa                  NaN
Et Grado Urb                               NaN
geometry                                  None
Name: 164, dtype: object

In [ ]:
# for df in lstdf:
#     df.loc[164, df.columns[-5:-1]] = [2.1, 'Medio-bassa', '5.001-10.000', 'Piccole città e sobborghi']
#     df.loc[164, 'geometry'] = confini[confini['Codice Com'] == 'A001168'].geometry.iloc[0]

In [143]:
for n, df in zip([2018, 2019, 2021], lstdf):
    gpd.GeoDataFrame(df).to_file(f'/nfs/home/genovese/thesis-wildfire-genovese/data/gathering/per_comune/IFC/{n}')

/tmp/ipykernel_1087574/2270595564.py:2: UserWarning: Column names longer than 10 characters will be truncated when saved to ESRI Shapefile.
  gpd.GeoDataFrame(df).to_file(f'/nfs/home/genovese/thesis-wildfire-genovese/data/gathering/per_comune/IFC/{n}')
/nfs/home/genovese/.venv/lib/python3.10/site-packages/pyogrio/raw.py:723: RuntimeWarning: Normalized/laundered field name: 'Codice comune' to 'Codice com'
  ogr_write(
/nfs/home/genovese/.venv/lib/python3.10/site-packages/pyogrio/raw.py:723: RuntimeWarning: Normalized/laundered field name: 'IFC (decili)' to 'IFC (decil'
  ogr_write(
/nfs/home/genovese/.venv/lib/python3.10/site-packages/pyogrio/raw.py:723: RuntimeWarning: Normalized/laundered field name: 'Classe ampiezza demografca 21 (estesa)' to 'Classe amp'
  ogr_write(
/nfs/home/genovese/.venv/lib/python3.10/site-packages/pyogrio/raw.py:723: RuntimeWarning: Normalized/laundered field name: 'Descrizione Decile' to 'Descrizion'
  ogr_write(
/nfs/home/genovese/.venv/lib/python3.10/site-p